In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ChromeDriver 경로 설정
chromedriver_path = r'C:\chromedriver\chromedriver.exe'

# WebDriver 초기화
chrome_options = Options()
chrome_options.binary_location = r'C:\Program Files\Google\Chrome\Application\chrome.exe'

# Chrome WebDriver 초기화
driver = webdriver.Chrome(service=Service(chromedriver_path), options=chrome_options)

# 홈페이지로 이동
driver.get("https://www.bigkinds.or.kr/v2/news/index.do")
time.sleep(2)

# 검색 옵션 설정
search_option_script = "document.querySelector('#collapse-step-1-body > div.srch-detail.v2 > div > div.tab-btn-wp1 > div.tab-btn-inner.tab1 > a').click();"
driver.execute_script(search_option_script)
time.sleep(2)

# 1개월 기간 선택 (라디오 버튼 직접 선택)
js_script = "document.querySelector('#srch-tab1 > div > div.radio-btn-wp.col6 > span:nth-child(3) > label').click();"
driver.execute_script(js_script)
time.sleep(2)

# 검색 시작 날짜 입력
begin_date_script = "document.querySelector('#search-begin-date').value = '2023-10-01';"
driver.execute_script(begin_date_script)
time.sleep(2)

# 검색 종료 날짜 입력
end_date_script = "document.querySelector('#search-end-date').value = '2023-11-01';"
driver.execute_script(end_date_script)
time.sleep(2)

# 사건사고 클릭
js_script = "document.querySelector('#collapse-step-1-body > div.srch-detail.v2 > div > div.tab-btn-wp2 > div.tab-btn-inner.tab4 > a').click();"
driver.execute_script(js_script)
time.sleep(2)

# expander 클릭
js_script = "document.querySelector('#srch-tab4 > ul > li:nth-child(4) > div > span.gj-tree-glyphicons-expander').click();"
driver.execute_script(js_script)
time.sleep(2)

# 사회갈등 클릭
js_script = "document.querySelector('#srch-tab4 > ul > li:nth-child(4) > ul > li:nth-child(2) > div > span:nth-child(4)').click();"
driver.execute_script(js_script)
time.sleep(2)

#적용하기
js_script = "document.querySelector('#search-foot-div > div.foot-btn > button.btn.btn-search.news-search-btn.news-report-search-btn').click();"
driver.execute_script(js_script)
time.sleep(2)

# "100건씩 보기" 옵션 선택
select_element = driver.find_element(By.ID, "select2")
options = select_element.find_elements(By.TAG_NAME, "option")
for option in options:
    if option.text == "100건씩 보기":
        option.click()
        break

# 페이지 로딩 대기
time.sleep(3)

# 초기화
news_list = []

# 뉴스 본문 가져오기
while True:
    for i in range(1, 101):
        try:
            # 뉴스 창 열기
            js_script = f"document.querySelector('#news-results > div:nth-child({i}) > div > div.cont > a > div').click();"
            driver.execute_script(js_script)
            time.sleep(1)

            # 기다릴 최대 시간: 10초
            wait = WebDriverWait(driver, 3)

            # 뉴스 본문 내용 가져오기
            news_content = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#news-detail-modal > div > div > div.modal-body > div > div.news-view-body'))).text

            # 뉴스 정보를 리스트에 추가
            news_list.append({'뉴스 본문': news_content})

            # 뉴스 창 닫기
            js_script = "document.querySelector('#news-detail-modal > div > div > div.modal-footer > button').click();"
            driver.execute_script(js_script)
            time.sleep(1)

        except Exception as e:
            break

    # 다음 페이지로 이동
    try:
        next_page_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#news-results-tab > div.data-result-ac.pc-only.paging-v2-wrp.btm > div.data-result-btm > div > div > div > div > div:nth-child(7) > a')))
        next_page_element.click()
        time.sleep(2)
    except Exception as e:
        break

# 뉴스 정보를 데이터프레임으로 만들기
df = pd.DataFrame(news_list)

# 데이터프레임을 엑셀 파일로 저장
df.to_excel('뉴스 본문만.xlsx', index=False)

# 웹 드라이버 종료
driver.quit()